# Data Collection
We limit our scope to the top 10 pools (in terms of TVL).

In [ ]:
from pprint import pprint
from gql import gql, Client
from gql.transport.aiohttp import AIOHTTPTransport

from graphql_helper import run_query

In [ ]:
uniswap_v3_subgraph_url = "https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3"

In [ ]:
most_liquid_pool_ids_query = """
{
  pools(first: 10, orderBy: liquidity, orderDirection: desc) {
    id
  }
}
"""

most_liquid_pool_ids = run_query(uniswap_v3_subgraph_url, most_liquid_pool_ids_query)
pprint(most_liquid_pool_ids)

In [ ]:
most_liquid_pool_addrs = list(map(lambda x: x["id"],most_liquid_pool_ids["data"]["pools"]))
most_liquid_pool_addrs


In [ ]:
async def get_pool_by_id(id: str):
    transport = AIOHTTPTransport(url=uniswap_v3_subgraph_url)

    async with Client(
        transport=transport,
        fetch_schema_from_transport=True,
    ) as session:
        query = gql(
            """
            query getPoolById($pool_addr: ID!) {
                pool(id: $pool_addr) {
                    tick
                    token0 {
                        symbol
                        id
                        decimals
                    }
                    token1 {
                        symbol
                        id
                        decimals
                    }
                    feeTier
                    sqrtPrice
                    liquidity
                }
            }
            """
        )
        params = {"pool_addr": id}
        result = await session.execute(query, variable_values=params)
        pprint(result)

for pool_addr in most_liquid_pool_addrs:
    await get_pool_by_id(pool_addr)